In [1]:
# ! pip install BackTranslation
# ! pip install benepar
# ! pip install git+https://github.com/PrithivirajDamodaran/Parrot_Paraphraser.git
# ! pip install expertai-nlapi==2.3.1

In [2]:
from googletrans import Translator

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import random
import pandas as pd
import csv

In [4]:
#stop words list
stop_words = stopwords.words('english')
random.seed(1)

In [5]:
from BackTranslation import BackTranslation
import spacy
import benepar
benepar.download('benepar_en3')
from benepar.spacy_plugin import BeneparComponent
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import tensorflow_hub as hub 
import numpy as np
import os, sys
from sklearn.metrics.pairwise import cosine_similarity
from parrot import Parrot
import torch
import warnings
from rake_nltk import Rake
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\AH20780\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
C:\ProgramData\Anaconda3\envs\anuj_research\lib\site-packages\benepar\spacy_plugin.py:7: FutureWarning: BeneparComponent and NonConstituentException have been moved to the benepar module. Use `from benepar import BeneparComponent, NonConstituentException` instead of benepar.spacy_plugin. The benepar.spacy_plugin namespace is deprecated and will be removed in a future version.
  warnings.warn(


In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AH20780\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
class Utils():
    
    # Constructor
    def __init__(self): 
        pass 
    
   
    '''
    Translate other language to english for a given sentance
    @in_word : Input text to be translated from detected text language to english
    @return - Translated text. In case language is not getting detected, method returns input text
    '''
    def translate_to_en(self, in_word):
        try:
            translator = Translator()
            t = translator.detect(in_word)
            if(t.lang=="tl" ): # Filipino
                translated = translator.translate(in_word, src='tl', dest='en')
                return translated.text
            if(t.lang=="hi" ): # Hindi
                translated = translator.translate(in_word, src='hi', dest='en')
                return translated.text
        except Exception as e:
            raise Exception("Error occured in method translate_to_en -->"+str(e))

        return in_word;    
       
    # Filipino
    def translate_to_tl(self, in_word):
        try:
            translator = Translator()
            t = translator.detect(in_word)
            if(t.lang=="en" ):
                translated = translator.translate(in_word, src='en', dest='tl')
                return translated.text
        except Exception as e:
            raise Exception("Error occured in method translate_to_tl -->"+str(e))
           
        return in_word; 
    
    # Hindi
    def translate_to_hi(self, in_word):
        try:
            translator = Translator()
            t = translator.detect(in_word)
            if(t.lang=="en" ):
                translated = translator.translate(in_word, src='en', dest='hi')
                return translated.text
        except Exception as e:
            raise Exception("Error occured in method translate_to_hi -->"+str(e))
            
        return in_word; 
    
    # mid is the language param
    def back_translation(self, in_word, mid):
        bt = in_word
        if( mid=='de'):
            bt= self.translate_to_tl(in_word)
        elif (mid=='fr'):
            bt= self.translate_to_hi(in_word) 
        return self.translate_to_en(bt)

In [8]:
# Test the translation utility
# utl = Utils()
# et = "Facebook chief executive officer Mark Zuckerberg on Thursday announced that the company he founded is rebranding itself as Meta."
# et = utl.back_translation(et, mid='fr')
# print(et)

In [9]:
CHATBOT_NAME = "ACUBED"

In [10]:
def full_backtranslation(txt):
    bt_sents=[]
    
    trans = BackTranslation(url=[
          'translate.google.com'
        ], proxies={'http': '127.0.0.1:1234', 'http://host.name': '127.0.0.1:4012'})
    result = trans.translate(txt, src='en')
    bt_sents.append(result.result_text)
    
    utls= Utils()
    et = utls.back_translation(txt,'hi')
    bt_sents.append(et)
    et = utls.back_translation(txt,'tl')
    bt_sents.append(et)
    return bt_sents
    

def nv_phrase_backtranslation(txt):
    nlp = spacy.load("en_core_web_sm") 
    if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
    else:
        nlp.add_pipe("benepar", config={"model": "benepar_en3"})
    doc = nlp(txt)
    sent = list(doc.sents)[0]
    print(f'BT String: {sent._.parse_string}')
    # (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
    print(f'BT String: {sent._.labels}')
    # ('S',)
    print(f'BT String: {list(sent._.children)[0]}')
    # The time for action


def get_synonyms(word):
    synonyms = set()
    
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

def synonym_replacement(words, n):
    words = words.lower()
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in words:
        if ele in punc:
            words = words.replace(ele, "")
    
    words = words.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n: # only replace up to n words
            break
    sentence = ' '.join(new_words)

    return sentence

def synonym_generator(words):
    words = words.lower()
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in words:
        if ele in punc:
            words = words.replace(ele, "")
    
    words = words.split()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        print(f'synonyms for "{random_word}": {synonyms}') 


def generate_paraphrase(txt): 
    retVal = []
    warnings.filterwarnings("ignore")
    
    ''' 
    uncomment to get reproducable paraphrase generations
    def random_state(seed):
      torch.manual_seed(seed)
      if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    random_state(1234)
    '''
        
    #Init models (make sure you init ONLY once if you integrate this to your code)
    parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)
    
    print("With default knobs:")
    para_phrases = parrot.augment(input_phrase=txt, use_gpu=False)
    for para_phrase in para_phrases:
        print(para_phrase)
        retVal.append(para_phrase[0])
    
    print("\nWith Diversity Ranker as Levenshtein:")
    para_phrases = parrot.augment(input_phrase=txt, use_gpu=False, diversity_ranker="levenshtein")
    for para_phrase in para_phrases:
        print(para_phrase)
        retVal.append(para_phrase[0])
    
    print("\nWith Do_diverse set:")
    para_phrases = parrot.augment(input_phrase=txt, use_gpu=False, do_diverse=True)
    for para_phrase in para_phrases:
        print(para_phrase)
        retVal.append(para_phrase[0])
        
    return retVal

In [11]:
def remove_stop_words(txt):
    example_sent =txt
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    
    return  filtered_sentence


def extract_theme_words(txt):
    retVal=[]
    rake_nltk_var = Rake()
    rake_nltk_var.extract_keywords_from_text(txt)
    keyword_extracted = rake_nltk_var.get_ranked_phrases()
    print(f'RAKE extracted keywords for understanding intention: {keyword_extracted}')

    tokenizer = RegexpTokenizer(r'\w+')
    
    # Single keywords of key words are being appended
    words_appended = []
    for key_words in keyword_extracted:
        tokens = tokenizer.tokenize(key_words)
        if (len(tokens) == 1):
            words_appended.append(tokens[0])
        else :
            retVal.append(key_words)
    
    if (words_appended):
        if ((len (words_appended) ==1) and retVal):
            # Single word appended contains one word, and its added to first keyword extract string
            retVal[0] = retVal[0] + ' '+ ' '.join(words_appended)
        else:
            retVal.append(' '.join(words_appended))    
        
    return retVal

In [12]:
# filter out top 2 similar sentence for the paraphrase_pool
def filter_duplication(txt, utterances):

    # get cosine similairty matrix
    def cos_sim(input_vectors):
        similarity = cosine_similarity(input_vectors)
        return similarity
    
 
    # get topN similar sentences
    def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
        # find the index of sentence in list
        index = sentence_list.index(sentence)
        # get the corresponding row in similarity matrix
        similarity_row = np.array(similarity_matrix[index, :])
        # get the indices of top similar
        indices = similarity_row.argsort()[-topN:][::-1]
        return [sentence_list[i] for i in indices]
        
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
      
    # Import the Universal Sentence Encoder's TF Hub module
    embed = hub.Module(module_url)
   
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)
   
    sentences_list = utterances
    
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentences_embeddings = session.run(embed(sentences_list))
    
    similarity_matrix = cos_sim(np.array(sentences_embeddings))
    
    sentence = txt
    top_similar = get_top_similar(sentence, sentences_list, similarity_matrix, 3)
   
    # printing the list using loop 
    for x in range(len(top_similar)): 
        print(top_similar[x])
    
    #get the index of a given item    
    indx = utterances.index(txt)
    print("indx=",indx)
    #remove item from list
    utterances.pop(indx)
    return utterances

In [13]:
def question_to_utterance (questionTxt): 
    retVal=[]
    print("Question Text-->",questionTxt)
    
    #add org text in the pool
    retVal.append(questionTxt)
    
    nv_phrase_backtranslation(questionTxt)
    
    a=full_backtranslation(questionTxt)
    print("Backtranslation Text-->",a)
    retVal.extend(a)
    
    a=extract_theme_words(questionTxt)
    print("Extracted Theme Words-->",a)
    retVal.extend(a)
     
    a = synonym_replacement(questionTxt, 2)
    print("Synonyms-->",a)
    retVal.extend([a])
    
    synonym_generator(questionTxt)
    
    print("\nParrot Paraphrase with Similarity Index-->")
    a=generate_paraphrase(questionTxt)
    print("\nParrot Paraphrase Extracted List-->",a)
    retVal.extend(a)
    
    
    #convert to lower case list followed by unique list
    retVal = [each_string.lower() for each_string in retVal]
    retVal = list(set(retVal))
    
    print("\nUtterances List-->",retVal)
    
    # Duplication removal - use only if required
#     retVal = filter_duplication(questionTxt.lower(), retVal)
#     print("Duplication Filtered Utterances-->",retVal)
     
    
    return retVal

In [14]:
def get_intent (questionTxt):
    
    questionTxt= questionTxt.replace(CHATBOT_NAME, "chatbot") # replace chatbot name as "chatbot"
    print(f'Question text--> {questionTxt}')
    intent= []
    
    with open('.env') as f:
        lines = f.readlines()
        username = lines[0].strip()
        password = lines[1].strip()
    os.environ["EAI_USERNAME"] = username
    os.environ["EAI_PASSWORD"] = password
    
    from expertai.nlapi.cloud.client import ExpertAiClient
    client = ExpertAiClient()
    language= 'en'
    document = client.specific_resource_analysis(
        body={"document": {"text": questionTxt}}, 
        params={'language': language, 'resource': 'relevants'})
    
 
    print('')  
    phrases={}
    for main_phrase in document.main_phrases:
        print (f'{main_phrase.value:{20}} {main_phrase.score:{5}}')
        phrases[main_phrase.value]=main_phrase.score
    
    print("Phrases==>",phrases)
    if phrases:
        max_key = max(phrases, key=phrases.get)
        intent.extend(max_key.split())
              
    print('')    
    concepts={}
    for main_concept in document.main_syncons:
        print (f'{main_concept.lemma:{20}} {main_concept.score:{5}}')
        concepts[main_concept.lemma]=main_concept.score
     
    print("Concepts==>",concepts)
    if concepts:
        max_key = max(concepts, key=concepts.get)
        intent.extend(max_key.split())

    print('')      
    lemmas={}
    for main_lemma in document.main_lemmas:
        print (f'{main_lemma.value:{20}} {main_lemma.score:{5}}')
        lemmas[main_lemma.value]=main_lemma.score
    
    print("Lemmas==>",lemmas)
    if lemmas:
        max_key = max(lemmas, key=lemmas.get)
        intent.extend(max_key.split())

    print("\n")
        
    print(f'Intent raw concepts: {intent}')
    intent_max_word=10
    return "_".join(list(dict.fromkeys(intent[:intent_max_word])))

In [15]:
def faq_csv_modinfire(csv_file_path):
        csvfile = open(csv_file_path,errors="backslashreplace")
        csvfileDf = pd.read_csv(csvfile, sep=',', quoting=csv.QUOTE_MINIMAL)
        csvfileDf = csvfileDf.fillna('')
        csvfileDf.columns = csvfileDf.columns.str.strip()
        
        #Create records from csvDF file
        for index, row in csvfileDf.iterrows():
            question= row['question']
            
            print("-"*100)
            print("\n")
            print("Question-->",question)
        
            print("\nUTTERANCE")
            utters_with_pipe="|".join(question_to_utterance(question))
            print("\nDone!  These are the final Utterances-->\n",utters_with_pipe)
            
            print("\nINTENT AND ENTITY")
            intent_txt= get_intent(question)
            print("Done! Intent is -->", intent_txt)
            print("-"*100)
            print("\n")
            
            
            
            # updating the column value/data
            csvfileDf.loc[index, 'utterances'] = utters_with_pipe
            csvfileDf.loc[index, 'intent'] = intent_txt
        
        # writing into the file
        csvfileDf.to_csv(csv_file_path.replace(".csv", "_output.csv"), index=False)
        print("Completed!")

In [16]:
%%capture cap --no-stderr
FAQ_CSV_INTENT_FILE = 'faqs_raw_input.csv'
faq_csv_modinfire(FAQ_CSV_INTENT_FILE)

C:\ProgramData\Anaconda3\envs\anuj_research\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [17]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)